In [1]:
num_sample = 300
num_burn = 20
sample_size = 800
n_cpu = 10

min_degree = 2
max_degree = 5

In [2]:
from dgp import sample_network_chain, get_graph, sample_Y1, sample_Y2, agcEffect
import numpy as np

# 1. Simulate or load data
adj = get_graph(sample_size, min_degree, max_degree, seed=1)
print(adj.sum(axis=1).min(), adj.sum(axis=1).max())

tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
rho = np.array([[0,0.1,0.2],
                [0.1,0,0.1],
                [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
nu = np.array([0.1,0,0,0.1,0,0,0.1,0,0]).reshape(3,3)       # shape (3, 3)
gamma = np.array([-1,2,0.1,-2,0.1,2,0.1,0.1])    # shape (8,)   
beta = np.array([-1,2,0.1,2,0.1,-2,0.1,2,0.1,0.1])

Y_chain, A_chain, L_chain = sample_network_chain(adj, tau, rho, nu, gamma, beta, R=num_sample,
    burnin_R=num_burn, seed=0, sample_Y_func=sample_Y1, Atype=('gen', 0.7))

Y_chain = Y_chain[::3]
A_chain = A_chain[::3]
L_chain = L_chain[::3]

dir, dir2, dir3, dir4 = [], [], [], []
for i in range(Y_chain.shape[0]):
    Y = Y_chain[i]
    A = A_chain[i]
    L = L_chain[i]
    dir.append(np.mean(np.mean(Y[A==1]) - Y[A==0]))
    dir2.append(np.mean(A))
    dir3.append(np.mean(A[L[:,0]==1]) - np.mean(A[L[:,0]==0]))
    dir4.append(np.mean(Y[L[:,0]==1]) - np.mean(Y[L[:,0]==0]))

np.mean(dir), np.mean(dir2), np.mean(Y_chain), np.mean(dir3), np.mean(dir4)

2 5


100%|██████████| 320/320 [00:00<00:00, 397.90it/s]


(0.557220354601125,
 0.4892375,
 0.639375,
 0.34131590515360005,
 0.3324466455259462)

In [3]:
agcEffect(
    adj,
    tau, rho, nu, beta,
    treatment_allocation=0.7,
    R=100,
    burnin_R=100,
    seed=0,
    sample_Y_func=sample_Y1
)

100%|██████████| 200/200 [00:00<00:00, 579.97it/s]


{'average': 0.743925,
 'direct_effect': 0.2950624999999997,
 'spillover_effect': 0.047375000000000056,
 'psi_1_gamma': 0.8341374999999998,
 'psi_0_gamma': 0.5390750000000001,
 'psi_zero': 0.4917}

In [4]:
from utils import run_pll
from run_pll import run_autognet

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_ag = run_pll(run_autognet, args, processes=n_cpu)
res_list_array_ag = np.array(res_list_ag)


Multiprocessing <function run_autognet at 0x13c5bcca0> in 100 tasks, with 10 processes...


100%|██████████| 60/60 [00:00<00:00, 526.35it/s]


Multiprocessing finished.


In [5]:
from run_pll import column_names

ret_mean_ag = res_list_array_ag.mean(axis=0)
ret_std_ag = res_list_array_ag.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_ag[i]:.5f} ± {ret_std_ag[i]:.5f}")

average: 0.74909 ± 0.03107
direct_effect: 0.29283 ± 0.04580
spillover_effect: 0.04936 ± 0.06525
psi_1_gamma: 0.83558 ± 0.03262
psi_0_gamma: 0.54275 ± 0.04714
psi_zero: 0.49339 ± 0.05638


In [6]:
from drnet import expit
from sklearn.linear_model import LogisticRegression

def get_prob(L, Y, A):
    x_a = [L[:, 0], np.zeros(L.shape[0]), 
           L[:, 1], np.zeros(L.shape[0]), 
           L[:, 2], np.zeros(L.shape[0]), 
           np.zeros(L.shape[0])]
    
    # train lostic regression
    model = LogisticRegression(penalty=None, solver='lbfgs', max_iter=10000)
    model.fit(np.array(x_a).T, A)
    
    gamma = np.zeros(8)
    gamma[0] = model.intercept_[0]
    gamma[1:] = model.coef_[0]

    linpred = (
            gamma[0]
            + gamma[1] * L[:, 0] + gamma[2] * 0
            + gamma[3] * L[:, 1] + gamma[4] * 0
            + gamma[5] * L[:, 2] + gamma[6] * 0
            + gamma[7] * 0
        )
    
    probs = expit(linpred)

    gamma_1 = (Y/probs*(A==1)).mean() 
    gamma_0 = (Y/(1-probs)*(A==0)).mean() 

    return gamma_1, gamma_0

res = np.array([get_prob(L_chain[i], Y_chain[i], A_chain[i]) for i in range(len(Y_chain))])
print(res.mean(axis=0))
print(res.std(axis=0))
# ret_tmp = get_prob(L_chain[-1], Y_chain[-1], A_chain[-1])
# ret_tmp

[0.83137892 0.52294249]
[0.02939118 0.04893953]


In [7]:
from drnet import *

ret_raw = doubly_robust(A_chain[-1], L_chain[-1], Y_chain[-1], adj_matrix=adj, return_raw=True)
print(ret_raw['psi_1_gamma'].mean(), ret_raw['psi_0_gamma'].mean())

0.8938885361746199 0.34527731222485997


In [8]:
from run_pll import *
from utils import run_pll

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i,
         'mispec': None} for i in range(len(Y_chain))]

res_list_dr = run_pll(run_dr, args, processes=n_cpu)
res_list_array_dr = np.array(res_list_dr)

from run_pll import column_names

ret_mean_dr = res_list_array_dr.mean(axis=0)
ret_std_dr = res_list_array_dr.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_dr[i]:.5f} ± {ret_std_dr[i]:.5f}")

Multiprocessing <function run_dr at 0x1183a04c0> in 100 tasks, with 10 processes...
Multiprocessing finished.
average: 0.74576 ± 0.04771
direct_effect: 0.35930 ± 0.08317
spillover_effect: 0.07362 ± 0.21426
psi_1_gamma: 0.85530 ± 0.06057
psi_0_gamma: 0.49600 ± 0.06966
psi_zero: 0.42238 ± 0.19960


In [9]:
from utils import run_pll
from run_pll import run_autognet

np.random.seed(0)
L_chain_noise = np.random.binomial(n=1, p=0.2, size=L_chain.shape)
args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain_noise,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_ag = run_pll(run_autognet, args, processes=n_cpu)
res_list_array_ag = np.array(res_list_ag)


Multiprocessing <function run_autognet at 0x13c5bcca0> in 100 tasks, with 10 processes...


100%|██████████| 60/60 [00:00<00:00, 526.84it/s]


Multiprocessing finished.


In [10]:
from run_pll import column_names

ret_mean_ag = res_list_array_ag.mean(axis=0)
ret_std_ag = res_list_array_ag.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_ag[i]:.5f} ± {ret_std_ag[i]:.5f}")

average: 0.77355 ± 0.01406
direct_effect: 0.55967 ± 0.06506
spillover_effect: 0.06982 ± 0.08436
psi_1_gamma: 0.93359 ± 0.01315
psi_0_gamma: 0.37392 ± 0.06128
psi_zero: 0.30410 ± 0.04758


In [11]:
# from run_pll import *
# from utils import run_pll

# args = [{'Y_chain': Y_chain,
#          'A_chain': A_chain,
#          'L_chain': L_chain,
#          'adj': adj,
#          'i': i,
#          'mispec': 'outcome'} for i in range(len(Y_chain))]

# res_list_dr = run_pll(run_dr, args, processes=n_cpu)
# res_list_array_dr = np.array(res_list_dr)

# from run_pll import column_names

# ret_mean_dr = res_list_array_dr.mean(axis=0)
# ret_std_dr = res_list_array_dr.std(axis=0)
# for i in range(len(column_names)):
#     print(f"{column_names[i]}: {ret_mean_dr[i]:.5f} ± {ret_std_dr[i]:.5f}")

In [12]:
# from run_pll import *
# from utils import run_pll

# args = [{'Y_chain': Y_chain,
#          'A_chain': A_chain,
#          'L_chain': L_chain,
#          'adj': adj,
#          'i': i,
#          'mispec': 'treatment'} for i in range(len(Y_chain))]

# res_list_dr = run_pll(run_dr, args, processes=n_cpu)
# res_list_array_dr = np.array(res_list_dr)

# from run_pll import column_names

# ret_mean_dr = res_list_array_dr.mean(axis=0)
# ret_std_dr = res_list_array_dr.std(axis=0)
# for i in range(len(column_names)):
#     print(f"{column_names[i]}: {ret_mean_dr[i]:.5f} ± {ret_std_dr[i]:.5f}")